In [23]:
# import tensorflow as tf 
# tensorflow 2.3 version 말고 1.0 version 사용
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior()

import numpy as np

In [24]:
tf.set_random_seed(777)

x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]

# one hot encoding으로 표현
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

In [25]:
X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3])

# 분류해야하는 클래스가 3
nb_classes = 3

W = tf.Variable(tf.random_normal([4,nb_classes]), name = 'weight')
b = tf.Variable(tf.random_normal([nb_classes]), name = 'bias')

# softmax 식
# tf.matmul(X,W) + b => logit, score 라고 부르기도 함.
hypothesis = tf.nn.softmax(tf.matmul(X,W) + b)

# cost/loss 함수 # Y는 one-hot으로 주어짐.
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis), axis=1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# 세션 초기화 하여 실행
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        _, cost_val = sess.run([optimizer, cost], feed_dict = {X:x_data, Y:y_data})
        
        if step % 500 == 0:
            print(step, cost_val)
    
    print('-----------------')    
    a = sess.run(hypothesis, feed_dict={X:[[1, 11, 7, 9]]})
    print(a, sess.run(tf.argmax(a,1))) # 가장 큰값을 1로 준다.
    
    print('-----------------')    
    b = sess.run(hypothesis, feed_dict={X:[[1, 3, 4, 3]]})
    print(b, sess.run(tf.argmax(b,1))) # 가장 큰값을 1로 준다.
    
    print('-----------------')    
    c = sess.run(hypothesis, feed_dict={X:[[1, 1, 0, 1]]})
    print(c, sess.run(tf.argmax(c,1))) # 가장 큰값을 1로 준다.
    
    print('-----------------')    
    abc_all = sess.run(hypothesis, feed_dict={X:[[1, 11, 7, 9],[1, 3, 4, 3], [1, 1, 0, 1]]})
    print(abc_all, sess.run(tf.argmax(abc_all,1))) # 가장 큰값을 1로 준다.

0 6.3635654
500 0.4084465
1000 0.24111578
1500 0.19025627
2000 0.15671113
-----------------
[[4.6992414e-03 9.9529254e-01 8.2206279e-06]] [1]
-----------------
[[0.8694534  0.12154147 0.00900514]] [0]
-----------------
[[1.3045859e-08 3.4179259e-04 9.9965823e-01]] [2]
-----------------
[[4.6992414e-03 9.9529254e-01 8.2206361e-06]
 [8.6945331e-01 1.2154150e-01 9.0051424e-03]
 [1.3045859e-08 3.4179259e-04 9.9965823e-01]] [1 0 2]


결과는 a=1, b=0, c=2로 예측할 수 있다. 
Fancy Softmax Classifier (cross_entropy, one_hot, reshape)

In [26]:
# x 데이터를 통해 어떤 동물인지 예측하는 예제 
tf.set_random_seed(777)

# 동물 데이터를 불러옴.
xy = np.loadtxt('../../../DeepLearningZeroToAll/DeepLearningZeroToAll-master/data-04-zoo.csv', delimiter =',', dtype=np.float32)

x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

(101, 16) (101, 1)


In [27]:
nb_classes = 7 # 0~6까지 숫자

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])

# 0~6까지 하나의 컬럼으로 구성된 값을 one-hot-encoding 실행
# one hot에 사용되는 Y 값은 무조건 0부터 시작 
Y_one_hot = tf.one_hot(Y, nb_classes)
print("one_hot:", Y_one_hot)

# one hot 작업만 하면 차원이 하나 더생기기 때문에(N차원 -> N+1차원), reshape 작업이 한번더 필요하다. 
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) # -1 : 앞에는 알아서..
print("reshape one_hot:", Y_one_hot)

one_hot: Tensor("one_hot_1:0", shape=(?, 1, 7), dtype=float32)
reshape one_hot: Tensor("Reshape_1:0", shape=(?, 7), dtype=float32)


In [28]:
# 모델 적용

W = tf.Variable(tf.random_normal([16, nb_classes]), name = 'weight')
b = tf.Variable(tf.random_normal([nb_classes]), name ='bias')

# score라고도 표현
logits = tf.matmul(X,W) +b

hypothesis = tf.nn.softmax(logits)

# 이미 있는 함수를 사용하여 cost/loss func을 구해보자.
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                                labels=tf.stop_gradient([Y_one_hot])))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        _, cost_val, acc_val = sess.run([optimizer, cost, accuracy], feed_dict={X: x_data, Y:y_data})
        
        if step % 100 == 0:
            print("step {:5}\tCost: {:.3f}\tAcc: {:.2f}".format(step, cost_val, acc_val))
            
    pred = sess.run(prediction, feed_dict= {X:x_data})
    
    for p,y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p==int(y), p, int(y)))
        
        
        


step     0	Cost: 6.723	Acc: 0.00
step   100	Cost: 0.636	Acc: 0.81
step   200	Cost: 0.418	Acc: 0.90
step   300	Cost: 0.312	Acc: 0.91
step   400	Cost: 0.247	Acc: 0.95
step   500	Cost: 0.202	Acc: 0.96
step   600	Cost: 0.170	Acc: 0.97
step   700	Cost: 0.146	Acc: 0.97
step   800	Cost: 0.128	Acc: 0.99
step   900	Cost: 0.114	Acc: 0.99
step  1000	Cost: 0.102	Acc: 0.99
step  1100	Cost: 0.093	Acc: 1.00
step  1200	Cost: 0.085	Acc: 1.00
step  1300	Cost: 0.079	Acc: 1.00
step  1400	Cost: 0.073	Acc: 1.00
step  1500	Cost: 0.068	Acc: 1.00
step  1600	Cost: 0.064	Acc: 1.00
step  1700	Cost: 0.060	Acc: 1.00
step  1800	Cost: 0.057	Acc: 1.00
step  1900	Cost: 0.054	Acc: 1.00
step  2000	Cost: 0.051	Acc: 1.00
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y:

예측값과 실제 Y 값은 거의 동일하게 보임.